An example of using the plasticc light curves to put lots of transients around a DDF location.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.plots as plots
from mafContrib import Plasticc_metric, plasticc_slicer
from lsst.sims.utils import _healbin
import healpy as hp
from lsst.sims.featureScheduler.surveys import generate_dd_surveys


/Users/yoachim/lsstp3/python/miniconda3-4.3.21/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/Users/yoachim/lsstp3/python/miniconda3-4.3.21/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
runName = 'baseline_1exp_pairsmix_10yrs'
#runName = 'roll_mod3_sdf0.20mixed_10yrs'
#runName = 'very_alt3_rm5illum20_10yrs'
#This next command assumes you have downloaded baseline2018a.db into the tutorials directory in this repo.
#Change this path according to your datbase and where it is saved.
opsdb = db.OpsimDatabase(runName+'.db')
outDir = 'temp'
resultsDb = db.ResultsDb(outDir=outDir)

In [3]:
ddf_surveys = generate_dd_surveys()
survey = ddf_surveys[0]

In [4]:
ddf_surveys[0].survey_name

'DD:ELAISS1'

In [ ]:
bundleList = []
slicer = plasticc_slicer(model='SNIa-normal', seed=42,
                                         ra_cen=survey.ra, dec_cen=survey.dec, radius=np.radians(3.))
metric = Plasticc_metric(metricName=survey.survey_name+' SNIa')
sql = ''
summary_stats = [metrics.MeanMetric(maskVal=0)]
plotFuncs = [plots.HealpixSkyMap()]
bundle = metricBundles.MetricBundle(metric, slicer, sql, runName=runName, summaryMetrics=summary_stats,
                         plotFuncs=plotFuncs)
bundleList.append(bundle)

In [ ]:
bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
bgroup = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll()

Querying database SummaryAllProps with no constraint for columns ['observationStartMJD', 'filter', 'fieldRA', 'fiveSigmaDepth', 'fieldDec'].


In [ ]:
bgroup.plotAll(closefigs=False)

In [ ]:
# bundleDict['DD:ELAISS1 SNIa_WellSampled'].metricValues
nside = 1024
data = _healbin(slicer.slicePoints['ra'], slicer.slicePoints['dec'],
                                   bundleDict['DD:ELAISS1 SNIa_WellSampled'].metricValues, nside=nside,
                                   reduceFunc=np.mean, fillVal=slicer.badval)

In [ ]:
plt.scatter(np.degrees(slicer.slicePoints['ra']), np.degrees(slicer.slicePoints['dec']), alpha=.5,c=bundleDict['DD:ELAISS1 SNIa_WellSampled'].metricValues)

In [ ]:
rot = (np.degrees(survey.ra), np.degrees(survey.dec))
hp.gnomview(data, rot=rot)

In [ ]:
np.where(data > 0)